# Practical Exam: House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


# Task 1

The team at RealAgents knows that the city that a property is located in makes a difference to the sale price.

Unfortuntately they believe that this isn't always recorded in the data.

Calculate the number of missing values of the `city`.

 - You should use the data in the file "house_sales.csv".

 - Your output should be an object `missing_city`, that contains the number of missing values in this column.

In [ ]:
# Use this cell to write your code for Task 1

import pandas as pd
# Loading the file
df = pd.read_csv('house_sales.csv')
# Analyzing the 'city' column
print(df['city'].value_counts())
# Creating the missing_city object
missing_city = 73 # as seen with value_counts

Silvertown    517
Teasdale      366
Poppleton     362
Riverford     182
--             73
Name: city, dtype: int64


# Task 2

Before you fit any models, you will need to make sure the data is clean.

The table below shows what the data should look like.

Create a cleaned version of the dataframe.

 - You should start with the data in the file "house_sales.csv".

 - Your output should be a dataframe named `clean_data`.

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [ ]:
# Use this cell to write your code for Task 2

# Loading libraries
import pandas as pd
import numpy as np
# Loading the dataframe
df = pd.read_csv('house_sales.csv')
# Uncovering missing values in disguise as '-', '--', '.'
df['city'].replace('--',np.nan,inplace=True)
df['house_type'].replace({'Det.':'Detached','Semi':'Semi-detached','Terr.':'Terraced'},inplace=True)
df['area'] = df['area'].str.replace(' sq.m.','').astype(float)
# Replacing missing values
fillna_dict = {'city':'Unknown',
               'sale_date':'2023-01-01',
               'months_listed':np.mean(df['months_listed']).round(1),
               'bedrooms':int(np.mean(df['bedrooms']).round(0)),
               'house_type':df['house_type'].mode()[0],
               'area':np.mean(df['area']).round(1)}
df.fillna(fillna_dict,inplace=True)
# Changing categories
df['city'] = pd.Categorical(df['city'], categories=['Silvertown','Teasdale','Poppleton','Riverford','Unknown'])
df['house_type'] = pd.Categorical(df['house_type'], categories=['Detached','Semi-detached','Terraced'])
# Creating clean_data
clean_data = df.copy()
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   house_id       1500 non-null   int64   
 1   city           1500 non-null   category
 2   sale_price     1500 non-null   int64   
 3   sale_date      1500 non-null   object  
 4   months_listed  1500 non-null   float64 
 5   bedrooms       1500 non-null   int64   
 6   house_type     1500 non-null   category
 7   area           1500 non-null   float64 
dtypes: category(2), float64(2), int64(3), object(1)
memory usage: 73.7+ KB


# Task 3

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price.

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`.

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`.

 - Your answers should be rounded to 1 decimal place.   

In [ ]:
# Use this cell to write your code for Task 3

# Loading the file
df = pd.read_csv('house_sales.csv')
# Group by bedrooms
price_by_rooms = df.groupby('bedrooms')['sale_price'].agg(['mean','var']).reset_index() # <- dont forget the reset_index() method!
# Renaming columns
price_by_rooms.rename(columns={'mean':'avg_price','var':'var_price'}, inplace=True)
# Rounding to 1 decimal
price_by_rooms['avg_price'] = price_by_rooms['avg_price'].round(1)
price_by_rooms['var_price'] = price_by_rooms['var_price'].round(1)

price_by_rooms

,bedrooms,avg_price,var_price
0,2,67076.4,5.652896e+08
1,3,154665.1,2.378289e+09
2,4,234704.6,1.725211e+09
3,5,301515.9,2.484328e+09
4,6,375741.3,3.924432e+09


# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [ ]:
# Use this cell to write your code for Task 4

# Importing libraries
from sklearn.linear_model import LinearRegression
# Reading files
train_data = pd.read_csv('train.csv',parse_dates=['sale_date'])
validation_data = pd.read_csv('validation.csv',parse_dates=['sale_date'])
# Getting dummies from dataframes
train_data = pd.get_dummies(train_data,columns=['house_type','city'],drop_first=True)
validation_data = pd.get_dummies(validation_data,columns=['house_type','city'],drop_first=True)
# Working the date column on train_data
train_data['year'] = train_data['sale_date'].dt.year
train_data['month'] = train_data['sale_date'].dt.month
train_data.drop('sale_date',axis=1,inplace=True)
# Working the date column on validation_data
validation_data['year'] = validation_data['sale_date'].dt.year
validation_data['month'] = validation_data['sale_date'].dt.month
validation_data.drop('sale_date',axis=1,inplace=True)

# Dividing the data
X = train_data.drop('sale_price', axis=1)
y = train_data['sale_price']
# Fitting the model and predicting
lin_reg = LinearRegression()
lin_reg.fit(X,y)
y_pred_lin_reg = lin_reg.predict(validation_data)
# Creating the required dataframe
base_result = pd.DataFrame({'house_id':validation_data['house_id'],'price':y_pred_lin_reg})
base_result

,house_id,price
0,1331375,121583.794619
1,1630115,303660.601418
2,1645745,384960.340265
3,1336775,125539.764584
4,1888274,270845.711874
5,1567679,277936.591676
6,1298770,332482.306680
7,1662892,-13513.286421
8,1514321,168528.437821
9,1795371,210673.723676


# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [ ]:
# Use this cell to write your code for Task 5

# Importing libraries
from sklearn.ensemble import RandomForestRegressor
# Reading files
train_data = pd.read_csv('train.csv',parse_dates=['sale_date'])
validation_data = pd.read_csv('validation.csv',parse_dates=['sale_date'])
# Getting dummies from dataframes
train_data = pd.get_dummies(train_data,columns=['house_type','city'],drop_first=True)
validation_data = pd.get_dummies(validation_data,columns=['house_type','city'],drop_first=True)
# Working the date column on train_data
train_data['year'] = train_data['sale_date'].dt.year
train_data['month'] = train_data['sale_date'].dt.month
train_data.drop('sale_date',axis=1,inplace=True)
# Working the date column on validation_data
validation_data['year'] = validation_data['sale_date'].dt.year
validation_data['month'] = validation_data['sale_date'].dt.month
validation_data.drop('sale_date',axis=1,inplace=True)

# Dividing the data
X = train_data.drop('sale_price', axis=1)
y = train_data['sale_price']
# Fitting the model and predicting
random_forest = RandomForestRegressor()
random_forest.fit(X,y)
y_pred_random_forest = random_forest.predict(validation_data)
# Creating the required dataframe
compare_result = pd.DataFrame({'house_id':validation_data['house_id'],'price':y_pred_random_forest})
compare_result

,house_id,price
0,1331375,79858.38
1,1630115,304787.69
2,1645745,405677.67
3,1336775,105848.87
4,1888274,264123.00
5,1567679,280659.95
6,1298770,344908.09
7,1662892,38432.98
8,1514321,174002.82
9,1795371,209515.34
